# Energy Dissipation Comparing ipynb

This python notebook was prepared in 22.07.2023 by Fatih Bahri AYDIN (https://github.com/fatihbahri), a graduate student from Gebze Technical University, within the scope of NOWADS project.

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import math

In [2]:
plots_save_dir = r"C:\Users\pc\OneDrive_GTU\OneDrive - Gebze Teknik Üniversitesi\Tez\Proje\Dissipation Projects\11 - Herbers et al. (2006) - (DUCK94)\IPYNBs\EDC\Plots"

This code get result excel files from result_loc directory and create a dataframe. This dataframe occurs energy dissipation differences between boundary location and the other observation locations. SWAN method create EDC of SWAN calculates and MEAS method create observation changes between locations.

file_nb will be the number of model, with for loop you can get all of the model results. model_nb will be the number of model. There are 4 different model type in this project. Those are 0 = "Default", 1 = "Default_TRIAD", 2 = "No_Dissipation", 3 = "No_Dissipation_TRIAD" respectively.

show_SWAN and show_MEAS methods show (or also save) the charts of EDC.

To save charts, change the plots_save_dir and use show_SWAN(save=True) or show_MEAS(save=True) methods below.

In [9]:
class EDC(object):
    def __init__(self, result_loc, models, file_nb, model_nb):
        self.result_loc = result_loc
        self.models = models
        self.file_nb = file_nb
        self.model_nb = model_nb
        self.model = os.listdir(self.result_loc)[self.file_nb]
        self.date = self.model[-8:]
        self.plots_save_dir = plots_save_dir
    ############################# To find distances between locations, changing "Location X" expression to x_distance_value #############################
    LOCATIONS = ["LOCATION 0", "LOCATION 0 (P01)", "LOCATION 1", "LOCATION 2", "LOCATION 3", "LOCATION 4", "LOCATION 5", "LOCATION 6", "LOCATION 7", "LOCATION 8", "LOCATION 9", "LOCATION 10", "LOCATION 11", "LOCATION 12", "LOCATION 13", "LOCATION 14", "LOCATION 15"]
    x_values = [124.94, 124.94, 135.01, 145.42, 160.77, 169.50, 190.2, 205.34, 220.23, 240.55, 252.03, 264.7, 320.37, 370.08, 398.39, 480.34, 884.11]
    ############################# To find distances between locations, changing "Location X" expression to x_distance_value #############################
    def find_dist(self, loc):
        self.loc = loc
        return self.x_values[self.LOCATIONS.index(self.loc)]
    def model_name(self):
        return self.model
    def date_(self):
        return self.date
    def create_SWAN_df(self):
        # open SWAN Result excel file and get values.
        os.chdir(self.result_loc + "/" + self.model)
        self.df = pd.read_excel(self.date +"_" + self.models[self.model_nb] + "_Model_Results.xlsx", header=1)
        self.SWAN = self.df.iloc[11:51, 3:4] # Frequency values
        self.SWAN.rename(columns = {'Unnamed: 3':'Frequency (Hz)'}, inplace = True)
        self.SWAN.reset_index(inplace=True, drop=True)
        # Density values
        for i in range(4,19):
            self.location_dens = self.df.iloc[11:51, i:i+1]
            self.location_dens.reset_index(inplace=True, drop=True)
            self.location_dens.columns = [self.location_dens.iloc[0:1, :].values.flatten()[0]]
            self.SWAN = pd.concat([self.SWAN, self.location_dens], axis=1)
        # Clear the data and show dataframe properly
        self.SWAN.dropna(axis=1, thresh=2, inplace=True)
        self.SWAN_Dens = self.SWAN.iloc[:, 1:]
        self.SWAN_Dens.columns = pd.MultiIndex.from_product([["Density (m^2/Hz)"], self.SWAN_Dens.columns])
        self.SWAN = self.SWAN.iloc[:, :1]
        self.SWAN.columns = pd.MultiIndex.from_product([["Frequency (Hz)"], self.SWAN.columns])
        self.SWAN = pd.concat([self.SWAN, self.SWAN_Dens], axis=1)
        self.SWAN.drop(index=self.SWAN.index[0], axis=0, inplace=True)
        self.SWAN.dropna(axis=1, thresh=2)
        self.SWAN.reset_index(drop=True, inplace=True)
        self.Boun_Loc = self.SWAN.iloc[:, -1:] # Boun_Loc (Boundary-deepest observation location) is the last column of the SWAN DF.
        self.SW = self.SWAN.iloc[:, 0:1]
        self.serie = []
        for j in range(self.SWAN.shape[1]-2):
            #print(str(Boun_Loc.columns[0][1]) + " - " + str(SWAN.iloc[0:,-1*(j+2):-1*(j+1)].columns[0][1]))
            self.serie=[]
            for t in range(len(self.Boun_Loc)):
                self.dist = self.find_dist(self.SWAN.iloc[0:,-1*(j+2):-1*(j+1)].columns[0][1]) - self.find_dist(self.Boun_Loc.columns[0][1]) # Subtraction of x-axis distances between two locations
                self.formula = (self.Boun_Loc.iloc[t:t+1,:].values.flatten()-self.SWAN.iloc[t:t+1, j:j+1].values.flatten())/(self.Boun_Loc.iloc[t:t+1,:].values.flatten()*self.dist) # Implementation of the EDC formula described at the end of the Notebook
                self.serie.append(self.formula[0]) # Saving the result obtained with the formula
            self.temp_df = pd.DataFrame({ str(str(self.Boun_Loc.columns[0][1]) + " - " + str(self.SWAN.iloc[0:,-1*(j+2):-1*(j+1)].columns[0][1])):self.serie}) # A temporary df was created to be saved together with the frequency values in column 0.
            self.SW = pd.concat([self.SW, self.temp_df], axis=1) # Concat the temporary df and frequencies df.
            return self.SW
    def show_SWAN(self, save=False):
        # In case of you only want to see/save charts, run the code line below. Unnecessary use can significantly extend the runtime of the code.
        #self.create_SWAN_df()
        self.save = save # For saving the chart
        # markers defines different marker types.
        self.markers = ["o--", "s--", "go--", "p--", "k-" , "c-", "bo--" , "ro--" , "c--", "r-", "b-", "+--", "m--", "k--" ,"*--"]
        plt.figure(figsize=(18, 9))
        for plot in range(1,self.SW.iloc[0:1].shape[1]):
            plt.plot(self.SW.iloc[:, 0:1], self.SW.iloc[:, plot], self.markers[plot] ,label=self.SW.columns[plot])
        plt.legend(loc='best')
        plt.title(self.date + " SWAN") # Title will be as "11010100 SWAN"
        plt.ylim(0, 0.5) # In case of a limitation on the Y-axis
        if self.save == True:
            os.chdir(self.plots_save_dir)
            plt.savefig('SWAN_plot' + self.date + '.png')
        plt.show()
    def create_MEAS_df(self):
        os.chdir(self.result_loc + "/" + self.model)
        self.df = pd.read_excel(self.date +"_" + self.models[self.model_nb] + "_Model_Results.xlsx", header=1)
        self.MEAS0 = self.df.iloc[65:267, 3:4]
        self.MEAS0.columns = ["Frequency (Hz)"]
        self.MEAS0.columns = pd.MultiIndex.from_product([["Frequency (Hz)"], self.MEAS0.columns])
        self.MEAS2 = self.df.iloc[65:267, 4:19]
        self.MEAS2.columns = [h for h in self.df.iloc[65:66, 4:19].values[0]]
        self.MEAS2.columns = pd.MultiIndex.from_product([["Density (m^2/Hz)"], self.MEAS2.columns])
        self.MEAS = pd.concat([self.MEAS0, self.MEAS2], axis=1).iloc[1:, :].reset_index(drop=True).dropna(thresh=2, axis=1)
        self.Boun_Loc2 = self.MEAS.iloc[:, -1:]
        self.MS = self.MEAS.iloc[:, 0:1]
        self.serie2 = []
        for j in range(self.MEAS.shape[1]-2):
            self.serie2=[]
            for t in range(len(self.Boun_Loc2)):
                self.dist2 = self.find_dist(self.MEAS.iloc[0:,-1*(j+2):-1*(j+1)].columns[0][1]) - self.find_dist(self.Boun_Loc2.columns[0][1])
                self.formula2 = (self.Boun_Loc2.iloc[t:t+1,:].values.flatten()-self.MEAS.iloc[t:t+1, j:j+1].values.flatten())/(self.Boun_Loc2.iloc[t:t+1,:].values.flatten()*self.dist2)
                self.serie2.append(self.formula2[0])
            self.temp_df2 = pd.DataFrame({ str(str(self.Boun_Loc2.columns[0][1]) + " - " + str(self.MEAS.iloc[0:,-1*(j+2):-1*(j+1)].columns[0][1])):self.serie2})
            self.MS = pd.concat([self.MS, self.temp_df2], axis=1)
        return self.MS
    def show_MEAS(self, save=False):
        # In case of you only want to see/save charts, run the code line below. Unnecessary use can significantly extend the runtime of the code.
        #self.create_MEAS_df()
        self.save = save
        self.markers = ["o--", "s--", "go--", "p--", "k-" , "c-", "bo--" , "ro--" , "c--", "r-", "b-", "+--", "m--", "k--" ,"*--"]
        plt.figure(figsize=(18, 9))
        for plot in range(1,self.MS.iloc[0:1].shape[1]):
            plt.plot(self.MS.iloc[:, 0:1], self.MS.iloc[:, plot], self.markers[plot] ,label=self.MS.columns[plot])
        plt.legend(loc='best')
        plt.title(self.date + " Observations")
        plt.ylim(0, 0.5)
        if self.save == True:
            os.chdir(self.plots_save_dir)
            plt.savefig('MEAS_plot' + self.date + '.png')
        plt.show()

In [4]:
result_loc = r"C:\Users\pc\OneDrive_GTU\OneDrive - Gebze Teknik Üniversitesi\Tez\Proje\DUCK94 SWAN Models OLD version\11_models"

In [5]:
range_value = len(os.listdir(result_loc))

In [6]:
file_nb = 0
model_nb = 0

In [7]:
models = ["Default", "Default_TRIAD", "No_Dissipation", "No_Dissipation_TRIAD"]

In [11]:
for m in range(range_value):
    file_nb = m
    e1 = EDC(result_loc, models, file_nb, model_nb)
    print(e1.date_())
    #print(e1.create_SWAN_df())
    e1.create_SWAN_df()
    e1.show_SWAN(False)
    e1.create_MEAS_df()
    e1.show_MEAS(False)

11010100
   (Frequency (Hz), Frequency (Hz))  LOCATION 14 - LOCATION 13
0                              0.04                   0.031339
1                             0.044                   0.034526
2                            0.0484                   0.034011
3                            0.0532                   0.034602
4                            0.0586                   0.028151
5                            0.0644                   0.010015
6                            0.0709                  -0.004087
7                            0.0779                   0.000338
8                            0.0857                   0.007743
9                            0.0943                   0.001820
10                           0.1037                  -0.003247
11                           0.1141                  -0.004871
12                           0.1255                  -0.007658
13                           0.1381                  -0.006713
14                           0.1519           

## Value of EDC 

EDC is a value that compares energy dissipation subtraction between boundary location (deepest observation location) and other locations. The formula involves distances between locations, so the EDC value is independent from distances between locations.

$$ \text{EDC}_{15-14} = \frac{E_{15} - E_{14}}{E_{15} \times D_{15-14}} $$

Where:

$$ {\text{EDC}_{15-14} \text{: Energy Dissipation Change between locations 15-14 (}    \frac{1}{m})} $$
$$ E_{15} \text{: Energy density value at location 15 (}m^2\text{/Hz).} $$
$$ E_{14} \text{: Energy density value at location 14 (}m^2\text{/Hz).} $$
$$ D_{15-14} \text{: Distance between location 15 and location 14 (m).} $$

Location 15 is the boundary condition location where the deepest sensor is located.
Location 14 and other locations are shallower than the location 15